# Poly-peptide Chain Statistics Example

Example demonstrating how to extract protein cahins from PDB entries. This example uses a flatMap function to transform a structure to its polymer chains.

## Imports

In [1]:
from pyspark import SparkConf, SparkContext
from mmtfPyspark.filters import polymerComposition
from mmtfPyspark.io import MmtfReader
from mmtfPyspark.mappers import structureToPolymerChains

## Configure Spark

In [2]:
conf = SparkConf().setMaster("local[*]") \
                  .setAppName("polypeptideCahinStats")
sc = SparkContext(conf = conf)

## Read in mmtf files, flatMap to polymer chains, filter by polymer composition, and get number of groups

In [3]:
path = "/home/marshuang80/PDB/full"

chainLengths = MmtfReader.readSequenceFile(path, sc) \
                         .flatMap(structureToPolymerChains(False, True)) \
                         .filter(polymerComposition(polymerComposition.AMINO_ACIDS_20)) \
                         .map(lambda t: t[1].num_groups) \
                         .cache()

## Print out poly-peptide chain statistics

In [5]:
print(f"Total number of chains: {chainLengths.count()}")
print(f"Total number of groups: {chainLengths.sum()}")
print(f"Min chain length: {chainLengths.min()}")
print(f"Mean chain length: {chainLengths.mean()}")
print(f"Max chain length: {chainLengths.max()}")

Total number of chains: 347885
Total number of groups: 82513205
Min chain length: 1
Mean chain length: 237.1852911163173
Max chain length: 3660


## Terminate Spark

In [6]:
sc.stop()